In [4]:

from textwrap import wrap
import shutil
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
import os, warnings
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import numpy as np
import random

print('TensorFlow Version ' + tf.__version__)


def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'


seed_everything()

import warnings

warnings.filterwarnings("ignore")

TensorFlow Version 2.10.0


In [5]:
image_size = 384
batch_size = 16
n_classes = 3
EPOCHS = 40

train_path = 'dataset/train'
valid_path = 'dataset/valid'
test_path = 'dataset/test'

classes = {0: "Dry",
           1: "Normal",
           2: "Wet"}

In [2]:
def data_augment(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float16)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float16)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float16)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float16)

    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    if p_spatial > .75:
        image = tf.image.transpose(image)

    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3)  # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2)  # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1)  # rotate 90º

    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower=.7, upper=1.3)
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)

    return image

In [8]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                                                samplewise_center=True,
                                                                samplewise_std_normalization=True,
                                                                dtype='float16')

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                                               samplewise_center=True,
                                                               samplewise_std_normalization=True,
                                                               dtype='float16')

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                                                samplewise_center=True,
                                                                samplewise_std_normalization=True,
                                                                dtype='float16')

train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    seed=1,
    color_mode='rgb',
    shuffle=True,
    class_mode='categorical')

# same directory as training data

valid_gen = train_datagen.flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    seed=1,
    color_mode='rgb',
    shuffle=False,
    class_mode='categorical',
)

test_gen = test_datagen.flow_from_directory(
    test_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    seed=1,
    color_mode='rgb',
    shuffle=False,
    class_mode='categorical',
)

Found 247 images belonging to 3 classes.
Found 12 images belonging to 3 classes.
Found 145 images belonging to 3 classes.


In [9]:
from sklearn.utils import class_weight

train_labels = []
for i in range(len(train_gen)):
    _, labels = train_gen[i]
    train_labels.extend(np.argmax(labels, axis=1))

# Compute class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
class Patches(L.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID',
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
plt.figure(figsize=(4, 4))
patch_size = 7  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2

x = train_gen.next()
image = x[0][0]

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(image_size, image_size)
)

patches = Patches(patch_size)(resized_image)
print(f'Image size: {image_size} X {image_size}')
print(f'Patch size: {patch_size} X {patch_size}')
print(f'Patches per image: {patches.shape[1]}')
print(f'Elements per patch: {patches.shape[-1]}')

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))

for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy().astype('uint8'))
    plt.axis('off')

In [ ]:
from vit_keras import vit

vit_model = vit.vit_b16(
    image_size=image_size,
    activation='softmax',
    pretrained=True,
    include_top=True,
    pretrained_top=False,
    classes=200)

In [ ]:
model = tf.keras.Sequential([
    vit_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.gelu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.gelu),
    tf.keras.layers.Dense(3, 'softmax')
],
    name='vision_transformer')

model.summary()

In [ ]:
class_weights_dict

In [ ]:
warnings.filterwarnings("ignore")

learning_rate = 1e-5

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2),
              metrics=['accuracy'])

STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = valid_gen.n // valid_gen.batch_size

early_stopping_callbacks = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=1)
filepath = "checkpoints/no-smoted-saved-model-{epoch:02d}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                                                               save_best_only=False, save_weights_only=False,
                                                               mode='auto', period=1)

History = model.fit(x=train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_gen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,
                    callbacks=[early_stopping_callbacks, model_checkpoint_callback],
                    class_weight=class_weights_dict)

model.save("models/model.h5")

In [ ]:
def plot_history(item):
    plt.plot(History.history[item], label=item)
    plt.plot(History.history["val_" + item], label="val_" + item)
    plt.xlabel("Epochs")
    plt.ylabel(item)
    plt.title("Train and Validation {} Over Epochs".format(item), fontsize=14)
    plt.legend()
    plt.grid()
    plt.show()


plot_history("loss")
plot_history("accuracy")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

predicted_classes = np.argmax(model.predict(test_gen, steps=test_gen.n // test_gen.batch_size + 1), axis=1)
true_classes = test_gen.classes
class_labels = list(test_gen.class_indices.keys())
confusionmatrix = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 10))
sns.heatmap(confusionmatrix, cmap='Blues', annot=True, cbar=True, fmt='g', xticklabels=class_labels,
            yticklabels=class_labels)

print(classification_report(true_classes, predicted_classes))